#DS2002 Project 2: Capstone

Nathan Geng (nyg5wvz)

Performing ETL and Structured Streaming on MySQL Database

## Section 1: Prereqisites

Import libraries

In [0]:
import os
import json
import pymongo
import pyspark.pandas as pd  # This uses Koalas that is included in PySpark version 3.2 or newer.
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

Instantiate global variables

In [0]:
# Azure MySQL connection information
jdbc_hostname = "nyg5wvz-mysql.mysql.database.azure.com"
jdbc_port = 3306
src_database = "classicmodels" # use classicmodels database

connection_properties = {
  "user" : "nyg5wvz",
  "password" : "Passw0rd123",
  "driver" : "org.mariadb.jdbc.Driver"
}

# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "Cluster0"
atlas_database_name = "classicmodels_dw"
atlas_user_name = "NathanGeng"
atlas_password = "Passw0rd123"

# Data Files (JSON) Information ###############################
dst_database = "classicmodels_dlh"
base_dir = "dbfs:/FileStore/ds2002-capstone"

database_dir = f"{base_dir}/{dst_database}"
data_dir = f"{base_dir}/source_data"
batch_dir = f"{data_dir}/batch"
stream_dir = f"{data_dir}/stream"

# Fact Orders stream and output directories
orders_stream_dir = f"{stream_dir}"
orders_output_bronze = f"{database_dir}/fact_orders/bronze"
orders_output_silver = f"{database_dir}/fact_orders/silver"
orders_output_gold   = f"{database_dir}/fact_orders/gold"

# Delete the Streaming Files ################################## 
dbutils.fs.rm(f"{database_dir}", True) 

# Delete the Database Files ###################################
dbutils.fs.rm(database_dir, True)


Out[94]: False

Define global functions

In [0]:
# ######################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
# ######################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.vaxwhhh.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)


    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

# ######################################################################################################################
# Use this Function to Create New Collections by Uploading JSON file(s) to the MongoDB Atlas server.
# ######################################################################################################################
def set_mongo_collection(user_id, pwd, cluster_name, db_name, src_file_path, json_files):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.vaxwhhh.mongodb.net/{db_name}"
    client = pymongo.MongoClient(mongo_uri)
    db = client[db_name]
    
    '''Read in a JSON file, and Use It to Create a New Collection'''
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(src_file_path, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    client.close()
    
    return result

Use Azure MySQL Database to get Date Dimenson and Products Table

In [0]:
%sql
DROP DATABASE IF EXISTS classicmodels_dlh CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS classicmodels_dlh
COMMENT "classic_models data lakehouse"
LOCATION "dbfs:/FileStore/ds2002-capstone"
WITH DBPROPERTIES (contains_pii = true, purpose = "DS-2002 Capstone");

Date Dimension Table

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://nyg5wvz-mysql.mysql.database.azure.com:3306/classicmodels",
  dbtable "dim_date",
  user "nyg5wvz",
  password "Passw0rd123"
)

In [0]:
%sql
USE DATABASE classicmodels_dlh;

CREATE OR REPLACE TABLE classicmodels_dlh.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/ds2002-capstone/classicmodels_dlh/dim_date"
AS SELECT * FROM view_date

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED classicmodels_dlh.dim_date;

col_name,data_type,comment
date_key,int,null
full_date,date,null
date_name,string,null
date_name_us,string,null
date_name_eu,string,null
day_of_week,int,null
day_name_of_week,string,null
day_of_month,int,null
day_of_year,int,null
weekday_weekend,string,null


Verify dim_date table was added to new database

In [0]:
%sql
SELECT * FROM classicmodels_dlh.dim_date LIMIT 5

date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,week_of_year,month_name,month_of_year,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


Products Table

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_products
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://nyg5wvz-mysql.mysql.database.azure.com:3306/classicmodels",
  dbtable "products",
  user "nyg5wvz",
  password "Passw0rd123"
)

In [0]:
%sql
USE DATABASE classicmodels_dlh;

CREATE OR REPLACE TABLE classicmodels_dlh.dim_products
COMMENT "Products Table"
LOCATION "dbfs:/FileStore/ds2002-capstone/classicmodels_dlh/dim_products"
AS SELECT * FROM view_products

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED classicmodels_dlh.dim_products;

col_name,data_type,comment
productCode,string,null
productName,string,null
productLine,string,null
productScale,string,null
productVendor,string,null
productDescription,string,null
quantityInStock,int,null
buyPrice,"decimal(10,2)",null
MSRP,"decimal(10,2)",null
,,


Customers and Order Details Tables

In [0]:
display(batch_dir)

'dbfs:/FileStore/ds2002-capstone/source_data/batch'

In [0]:
source_dir = 'source_data/batch'

json_files = {"customers" : 'dim_customers.json', "order_details" : 'dim_order_details.json'}

set_mongo_collection(atlas_user_name, atlas_password, atlas_cluster_name, atlas_database_name, source_dir, json_files) 

Out[107]: <pymongo.results.InsertManyResult at 0x7f7fbb0a6e40>

Customers Table

In [0]:
%scala
import com.mongodb.spark._

val df_customers = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("database", "classicmodels_dw").option("collection", "customers").load()
.select("customerNumber", "customerName", "contactLastName", "contactFirstName", "phone", "addressLine1", "addressLine2", "city", "state", "postalCode", "country", "creditLimit")

display(df_customers)


customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,creditLimit
103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,21000
112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,71800
114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,117300
119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,118200
121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,81700
124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,,San Rafael,CA,97562,USA,210500
125,Havel & Zbyszek Co,Piestrzeniewicz,Zbyszek,(26) 642-7555,ul. Filtrowa 68,,Warszawa,,01-012,Poland,0
128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,,Frankfurt,,60528,Germany,59700
129,Mini Wheels Co.,Murphy,Julie,6505555787,5557 North Pendale Street,,San Francisco,CA,94217,USA,64600
131,Land of Toys Inc.,Lee,Kwai,2125557818,897 Long Airport Avenue,,NYC,NY,10022,USA,114900


In [0]:
%scala
df_customers.printSchema()

root
-- customerNumber: integer (nullable = true)
-- customerName: string (nullable = true)
-- contactLastName: string (nullable = true)
-- contactFirstName: string (nullable = true)
-- phone: string (nullable = true)
-- addressLine1: string (nullable = true)
-- addressLine2: string (nullable = true)
-- city: string (nullable = true)
-- state: string (nullable = true)
-- postalCode: string (nullable = true)
-- country: string (nullable = true)
-- creditLimit: double (nullable = true)

In [0]:
%scala
df_customers.write.format("delta").mode("overwrite").saveAsTable("classicmodels_dlh.dim_customer")

In [0]:
%sql
DESCRIBE EXTENDED classicmodels_dlh.dim_customer

col_name,data_type,comment
customerNumber,int,null
customerName,string,null
contactLastName,string,null
contactFirstName,string,null
phone,string,null
addressLine1,string,null
addressLine2,string,null
city,string,null
state,string,null
postalCode,string,null


In [0]:
%sql
SELECT * FROM classicmodels_dlh.dim_customer LIMIT 5

customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,creditLimit
103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,21000
112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,71800
114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,117300
119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,118200
121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,81700


### Section 3.2: Order Details Table
Read orders table from MongoDB

In [0]:
%scala
import com.mongodb.spark._

val df_order_details = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("database", "classicmodels_dw").option("collection", "order_details").load()
.select("orderNumber", "productCode", "quantityOrdered", "priceEach", "orderLineNumber")

display(df_order_details)

orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
10100,S18_1749,30,136,3
10100,S18_2248,50,55.09,2
10100,S18_4409,22,75.46,4
10100,S24_3969,49,35.29,1
10101,S18_2325,25,108.06,4
10101,S18_2795,26,167.06,1
10101,S24_1937,45,32.53,3
10101,S24_2022,46,44.35,2
10102,S18_1342,39,95.55,2
10102,S18_1367,41,43.13,1


In [0]:
%scala
df_order_details.printSchema()

root
-- orderNumber: integer (nullable = true)
-- productCode: string (nullable = true)
-- quantityOrdered: integer (nullable = true)
-- priceEach: double (nullable = true)
-- orderLineNumber: integer (nullable = true)

In [0]:
%scala
df_order_details.write.format("delta").mode("overwrite").saveAsTable("classicmodels_dlh.dim_order_details")

In [0]:
%sql
DESCRIBE EXTENDED classicmodels_dlh.dim_order_details

col_name,data_type,comment
id,int,null
company,string,null
last_name,string,null
first_name,string,null
job_title,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,classicmodels_dlh,
Table,dim_supplier,


In [0]:
%sql
SELECT * FROM classicmodels_dlh.dim_order_details LIMIT 5

orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
10100,S18_1749,30,136,3
10100,S18_2248,50,55.09,2
10100,S18_4409,22,75.46,4
10100,S24_3969,49,35.29,1
10101,S18_2325,25,108.06,4


Structured Streaming Fact Orders Table

In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")

 .option("cloudFiles.schemaHints", "orderNumber BIGINT")
 .option("cloudFiles.schemaHints", "orderDate DATE")
 .option("cloudFiles.schemaHints", "requiredDate DATE")
 .option("cloudFiles.schemaHints", "shippedDate DATE") 
 .option("cloudFiles.schemaHints", "phone STRING")
 .option("cloudFiles.schemaHints", "status STRING")
 .option("cloudFiles.schemaHints", "customerNumber BIGINT")

 .option("cloudFiles.schemaLocation", orders_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(orders_stream_dir)
 .createOrReplaceTempView("orders_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW orders_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM orders_raw_tempview
)

In [0]:
%sql
SELECT * FROM orders_bronze_tempview LIMIT 5

comments,customerNumber,orderDate,orderNumber,requiredDate,shippedDate,status,_rescued_data,receipt_time,source_file
,363,2003-01-06,10100,2003-01-13,2003-01-10,Shipped,null,2023-04-30T02:35:32.406+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json
Check on availability.,128,2003-01-09,10101,2003-01-18,2003-01-11,Shipped,null,2023-04-30T02:35:32.406+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json
,181,2003-01-10,10102,2003-01-18,2003-01-14,Shipped,null,2023-04-30T02:35:32.406+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json
,121,2003-01-29,10103,2003-02-07,2003-02-02,Shipped,null,2023-04-30T02:35:32.406+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json
,141,2003-01-31,10104,2003-02-09,2003-02-01,Shipped,null,2023-04-30T02:35:32.406+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json


Bronze Table

In [0]:
(spark.table("orders_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{orders_output_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_orders_bronze"))

Out[115]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f7fbb0645b0>

Create a Silver Table

In [0]:
(spark.readStream
  .table("fact_orders_bronze")
  .createOrReplaceTempView("orders_silver_tempview"))

In [0]:
%sql
SELECT * FROM orders_silver_tempview LIMIT 5

comments,customerNumber,orderDate,orderNumber,requiredDate,shippedDate,status,_rescued_data,receipt_time,source_file
,363,2003-01-06,10100,2003-01-13,2003-01-10,Shipped,null,2023-04-30T02:35:38.351+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json
Check on availability.,128,2003-01-09,10101,2003-01-18,2003-01-11,Shipped,null,2023-04-30T02:35:38.351+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json
,181,2003-01-10,10102,2003-01-18,2003-01-14,Shipped,null,2023-04-30T02:35:38.351+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json
,121,2003-01-29,10103,2003-02-07,2003-02-02,Shipped,null,2023-04-30T02:35:38.351+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json
,141,2003-01-31,10104,2003-02-09,2003-02-01,Shipped,null,2023-04-30T02:35:38.351+0000,dbfs:/FileStore/ds2002-capstone/source_data/stream/dim_orders.json


In [0]:
%sql
DESCRIBE EXTENDED orders_silver_tempview

col_name,data_type,comment
comments,string,null
customerNumber,bigint,null
orderDate,string,null
orderNumber,bigint,null
requiredDate,string,null
shippedDate,string,null
status,string,null
_rescued_data,string,null
receipt_time,timestamp,null
source_file,string,null


Creating a Fact Table

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_orders_silver_tempview AS (
  SELECT o.orderNumber,
  od.productCode,
  p.productName,
  p.productLine,
  p.productScale,
  p.productVendor,
  p.quantityInStock,
  p.buyPrice,
  p.MSRP,

  od.quantityOrdered,
  od.priceEach,
  od.orderLineNumber,

  o.customerNumber,
  c.customerName,
  c.city,
  c.state,
  c.country,
  c.creditLimit,

  o.orderDate,
  odate.day_name_of_week AS order_day_name_of_week,
  odate.day_of_month AS order_day_of_month,
  odate.weekday_weekend AS order_weekday_weekend,
  odate.month_name AS order_month_name,
  odate.calendar_quarter AS order_calendar_quarter,
  odate.calendar_year AS order_calendar_year,

  o.requiredDate,
  rd.day_name_of_week AS required_day_name_of_week,
  rd.day_of_month AS required_day_of_month,
  rd.weekday_weekend AS required_weekday_weekend,
  rd.month_name AS required_month_name,
  rd.calendar_quarter AS required_calendar_quarter,
  rd.calendar_year AS required_calendar_year,

  sd.date_key,
  o.shippedDate,
  sd.day_name_of_week AS shipped_day_name_of_week,
  sd.day_of_month AS shipped_day_of_month,
  sd.weekday_weekend AS shipped_weekday_weekend,
  sd.month_name AS shipped_month_name,
  sd.calendar_quarter AS shipped_calendar_quarter,
  sd.calendar_year AS shipped_calendar_year,

  o.status

  FROM orders_silver_tempview AS o
  INNER JOIN classicmodels_dlh.dim_customer AS c
  ON c.customerNumber = o.customerNumber
  INNER JOIN classicmodels_dlh.dim_order_details AS od
  ON od.orderNumber = o.orderNumber
  INNER JOIN classicmodels_dlh.dim_products AS p
  ON p.productCode = od.productCode
  LEFT OUTER JOIN classicmodels_dlh.dim_date AS odate
  ON odate.full_date = o.orderDate
  LEFT OUTER JOIN classicmodels_dlh.dim_date AS rd
  ON rd.full_date = o.requiredDate
  LEFT OUTER JOIN classicmodels_dlh.dim_date AS sd
  ON sd.full_date = o.shippedDate
)

In [0]:
%sql
SELECT * FROM fact_orders_silver_tempview LIMIT 5

orderNumber,productCode,productName,productLine,productScale,productVendor,quantityInStock,buyPrice,MSRP,quantityOrdered,priceEach,orderLineNumber,customerNumber,customerName,city,state,country,creditLimit,orderDate,order_day_name_of_week,order_day_of_month,order_weekday_weekend,order_month_name,order_calendar_quarter,order_calendar_year,requiredDate,required_day_name_of_week,required_day_of_month,required_weekday_weekend,required_month_name,required_calendar_quarter,required_calendar_year,date_key,shippedDate,shipped_day_name_of_week,shipped_day_of_month,shipped_weekday_weekend,shipped_month_name,shipped_calendar_quarter,shipped_calendar_year,status
10100,S24_3969,1936 Mercedes Benz 500k Roadster,Vintage Cars,1:24,Red Start Diecast,2081,21.75,41.03,49,35.29,1,363,Online Diecast Creations Co.,Nashua,NH,USA,114200,2003-01-06,Monday,6,Weekday,January,1,2003,2003-01-13,Monday,13,Weekday,January,1,2003,20030110,2003-01-10,Friday,10,Weekday,January,1,2003,Shipped
10100,S18_4409,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars,1:18,Exoto Designs,6553,43.26,92.03,22,75.46,4,363,Online Diecast Creations Co.,Nashua,NH,USA,114200,2003-01-06,Monday,6,Weekday,January,1,2003,2003-01-13,Monday,13,Weekday,January,1,2003,20030110,2003-01-10,Friday,10,Weekday,January,1,2003,Shipped
10100,S18_2248,1911 Ford Town Car,Vintage Cars,1:18,Motor City Art Classics,540,33.30,60.54,50,55.09,2,363,Online Diecast Creations Co.,Nashua,NH,USA,114200,2003-01-06,Monday,6,Weekday,January,1,2003,2003-01-13,Monday,13,Weekday,January,1,2003,20030110,2003-01-10,Friday,10,Weekday,January,1,2003,Shipped
10100,S18_1749,1917 Grand Touring Sedan,Vintage Cars,1:18,Welly Diecast Productions,2724,86.70,170.00,30,136,3,363,Online Diecast Creations Co.,Nashua,NH,USA,114200,2003-01-06,Monday,6,Weekday,January,1,2003,2003-01-13,Monday,13,Weekday,January,1,2003,20030110,2003-01-10,Friday,10,Weekday,January,1,2003,Shipped
10101,S24_2022,1938 Cadillac V-16 Presidential Limousine,Vintage Cars,1:24,Classic Metal Creations,2847,20.61,44.80,46,44.35,2,128,"Blauer See Auto, Co.",Frankfurt,,Germany,59700,2003-01-09,Thursday,9,Weekday,January,1,2003,2003-01-18,Saturday,18,Weekend,January,1,2003,20030111,2003-01-11,Saturday,11,Weekend,January,1,2003,Shipped


In [0]:
(spark.table("fact_orders_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{orders_output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_orders_silver"))

Out[140]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f7fbae2a1c0>

In [0]:
%sql
SELECT * FROM fact_orders_silver LIMIT 5

orderNumber,productCode,productName,productLine,productScale,productVendor,quantityInStock,buyPrice,MSRP,quantityOrdered,priceEach,orderLineNumber,customerNumber,customerName,city,state,country,creditLimit,orderDate,order_day_name_of_week,order_day_of_month,order_weekday_weekend,order_month_name,order_calendar_quarter,order_calendar_year,requiredDate,required_day_name_of_week,required_day_of_month,required_weekday_weekend,required_month_name,required_calendar_quarter,required_calendar_year,shippedDate,shipped_day_name_of_week,shipped_day_of_month,shipped_weekday_weekend,shipped_month_name,shipped_calendar_quarter,shipped_calendar_year,status
10100,S24_3969,1936 Mercedes Benz 500k Roadster,Vintage Cars,1:24,Red Start Diecast,2081,21.75,41.03,49,35.29,1,363,Online Diecast Creations Co.,Nashua,NH,USA,114200,2003-01-06,Monday,6,Weekday,January,1,2003,2003-01-13,Monday,13,Weekday,January,1,2003,2003-01-10,Friday,10,Weekday,January,1,2003,Shipped
10100,S18_4409,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars,1:18,Exoto Designs,6553,43.26,92.03,22,75.46,4,363,Online Diecast Creations Co.,Nashua,NH,USA,114200,2003-01-06,Monday,6,Weekday,January,1,2003,2003-01-13,Monday,13,Weekday,January,1,2003,2003-01-10,Friday,10,Weekday,January,1,2003,Shipped
10100,S18_2248,1911 Ford Town Car,Vintage Cars,1:18,Motor City Art Classics,540,33.30,60.54,50,55.09,2,363,Online Diecast Creations Co.,Nashua,NH,USA,114200,2003-01-06,Monday,6,Weekday,January,1,2003,2003-01-13,Monday,13,Weekday,January,1,2003,2003-01-10,Friday,10,Weekday,January,1,2003,Shipped
10100,S18_1749,1917 Grand Touring Sedan,Vintage Cars,1:18,Welly Diecast Productions,2724,86.70,170.00,30,136,3,363,Online Diecast Creations Co.,Nashua,NH,USA,114200,2003-01-06,Monday,6,Weekday,January,1,2003,2003-01-13,Monday,13,Weekday,January,1,2003,2003-01-10,Friday,10,Weekday,January,1,2003,Shipped
10101,S24_2022,1938 Cadillac V-16 Presidential Limousine,Vintage Cars,1:24,Classic Metal Creations,2847,20.61,44.80,46,44.35,2,128,"Blauer See Auto, Co.",Frankfurt,,Germany,59700,2003-01-09,Thursday,9,Weekday,January,1,2003,2003-01-18,Saturday,18,Weekend,January,1,2003,2003-01-11,Saturday,11,Weekend,January,1,2003,Shipped


In [0]:

%sql
DESCRIBE EXTENDED classicmodels_dlh.fact_orders_silver

col_name,data_type,comment
orderNumber,bigint,null
productCode,string,null
productName,string,null
productLine,string,null
productScale,string,null
productVendor,string,null
quantityInStock,int,null
buyPrice,"decimal(10,2)",null
MSRP,"decimal(10,2)",null
quantityOrdered,int,null


Gold Table

In [0]:
%sql
SELECT fo.country
, fo.productName
, fo.productLine
, SUM(fo.quantityOrdered) AS total_quantity_ordered
, fo.status

FROM classicmodels_dlh.fact_orders_silver AS fo
GROUP BY fo.country, fo.productLine, fo.productName, fo.status
ORDER BY total_quantity_ordered DESC

country,productName,productLine,total_quantity_ordered,status
USA,1957 Chevy Pickup,Trucks and Buses,523,Shipped
USA,2002 Suzuki XREO,Motorcycles,499,Shipped
USA,1992 Ferrari 360 Spider red,Classic Cars,491,Shipped
USA,1936 Chrysler Airflow,Vintage Cars,487,Shipped
USA,1997 BMW R 1100 S,Motorcycles,475,Shipped
USA,1998 Chrysler Plymouth Prowler,Classic Cars,466,Shipped
USA,1996 Moto Guzzi 1100i,Motorcycles,466,Shipped
USA,1969 Harley Davidson Ultimate Chopper,Motorcycles,464,Shipped
USA,1958 Chevy Corvette Limited Edition,Classic Cars,456,Shipped
USA,1917 Maxwell Touring Car,Vintage Cars,449,Shipped


Databricks visualization. Run in Databricks to view.

Data Visualization:
This graph shows the average number of products ordered by each country, in alphabetical order.

In [0]:
%sql
SELECT fo.country
, fo.productName
, SUM(fo.quantityOrdered) AS total_quantity_ordered

FROM classicmodels_dlh.fact_orders_silver AS fo
GROUP BY fo.country, fo.productName
ORDER BY total_quantity_ordered DESC

country,productName,total_quantity_ordered
USA,1957 Chevy Pickup,523
USA,2002 Suzuki XREO,499
USA,1992 Ferrari 360 Spider red,491
USA,1936 Chrysler Airflow,487
USA,1997 BMW R 1100 S,475
USA,1996 Moto Guzzi 1100i,466
USA,1998 Chrysler Plymouth Prowler,466
USA,1969 Harley Davidson Ultimate Chopper,464
USA,1958 Chevy Corvette Limited Edition,456
USA,1917 Maxwell Touring Car,449


Databricks visualization. Run in Databricks to view.